In [1]:
from data_load import get_dataloader
from model import TextCnn,TextRnn
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim


In [2]:
lr = 0.05
epoches = 50
class_num = 5
batch_size = 32
sentence_len = 50

model_list = ["rnn", "cnn"]

train_loader, test_loader, embed_pretrain, vocab_size = get_dataloader()
# model = TextRnn(vocab_size=vocab_size, embedding_dim=50, hidden_size=128, class_num=class_num, embedding_pretrained = embed_pretrain)
        # if model_type == "cnn": model = TextCnn(vocab_size=vocab_size, embedding_dim=50, hidden_size=128,
        # class_num=class_num,  embedding_pretrained=embed_pretrain)

  3%|▎         | 5252/156060 [00:00<00:02, 51822.58it/s]

Construct Corpus...


  8%|▊         | 12212/156060 [00:00<00:01, 122116.04it/s]

Convert Corpus to Integers
Convert Phrase to Integers


100%|██████████| 156060/156060 [00:00<00:00, 227694.79it/s]


loading vec embeding models...


  0%|          | 0/16532 [00:00<?, ?it/s]/remote-home/zyfei/nlp-beginner/task2/data_load.py:82: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629403081/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  res[idx] = torch.tensor(model.get_vector(word))
 14%|█▍        | 2390/16532 [00:00<00:00, 23888.52it/s]

convert models...


100%|██████████| 16532/16532 [00:00<00:00, 33414.79it/s]


total 16532 words and in glove find 15248 words


In [3]:
model

NameError: name 'model' is not defined

In [4]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

model = TextRnn(vocab_size=vocab_size, embedding_dim=50, class_num=5,hidden_size=128, embedding_pretrained = embed_pretrain, device=device)

optim = optim.Adam(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()

loss_history = []
train_acc_history = []
test_acc_history = []

model.to(device)

model.train()
losses = []
count = 0
for idx,(x,y) in enumerate(train_loader):
    # if len(x) < batch_size:
    #     break
    x,y = x.to(device),y.to(device)
    
    pred = model(x)
    optim.zero_grad()
    loss = loss_function(pred,y)
    loss.backward()
    losses.append(loss.item())
    optim.step()
train_loss = np.mean(losses)
loss_history.append(losses)

In [5]:
train_loss

1.6196707685992144

In [6]:
model.eval()

test_acces = []
for idx,(x,y) in enumerate(test_loader):
    x = x.to(device)
    y = y.numpy()
    y_pred = model(x).cpu()
    # print(y_pred)
    y_pred = np.argmax(y_pred.detach().numpy(),1)
    # print(y_pred)
    acc = np.mean(y_pred == y, dtype=np.float)
    test_acces.append(acc)
test_acc = np.mean(test_acces)
test_acc_history.append(test_acces)

In [8]:
test_acc

0.5004482581967213

In [7]:
train_loss

5.070047128053887

In [9]:
model = TextRnn(vocab_size=vocab_size, embedding_dim=50, class_num=5,hidden_size=128, embedding_pretrained = embed_pretrain)

In [10]:
model


TextRnn(
  (embedding_layer): Embedding(16532, 50, padding_idx=50)
  (rnn_layer): RNN(50, 128, batch_first=True)
  (linear_layer): Linear(in_features=128, out_features=5, bias=True)
)